In [2]:
import pandas as pd 
import re 
import numpy as np 
import ast
from shapely.geometry import shape
import folium 


**CLEANING DATAFRAME**

In [ ]:
df = pd.read_csv("data/scraped_results.csv")

In [4]:
df.head()

,brand,id,metadata.creationDate,metadata.updateDate,sections.location.address.city,sections.location.address.zipCode,sections.location.address.country,sections.location.geometry.type,sections.location.geometry.coordinates,sections.description.description,sections.description.headline,sections.hardFacts.title,sections.hardFacts.keyfacts,sections.hardFacts.price.value,numberOfRooms,livingSpace,numberOfFloors,numberOfBedrooms,availability,plotSpace
0,seloger,25EC9GDFBT55,2025-11-20T08:56:00Z,2025-11-23T01:16:54.498Z,Paris 6ème,75006,FRA,Point,"[2.3387919, 48.8536809]",## Space\r\n\r\nThis accommodation is availabl...,Appartement de 34m2,Appartement à louer,"['1 pièce', '34 m²', '1er étage']",2 800 €,1 pièce,34 m²,1er étage,NaN,NaN,NaN
1,seloger,2524YZ9M2KYE,2025-11-06T14:14:00Z,2025-11-19T17:36:08.608Z,Lyon 5ème,69005,FRA,Point,"[4.794783, 45.758385]","26 B, rue des Aqueducs 69005 Lyon, au cœur du ...","Location F4 - 26 B, rue des Aqueducs 69005 Lyon",Appartement à louer,"['4 pièces', '3 chambres', '91,7 m²', 'Étage 7...",1 666 €,4 pièces,"91,7 m²",Étage 7/7,3 chambres,NaN,NaN
2,seloger,24KPL52W4HXP,2024-03-05T14:14:00Z,2025-10-30T01:20:07.218Z,Lyon 6ème,69006,FRA,Point,"[4.844802, 45.768051]",Au coeur du quartier FOCH - au 17 rue de Sèze ...,Appartement 3 pièces,Appartement à louer,"['3 pièces', '2 chambres', '63,5 m²', '4ème ét...",1 440 €,3 pièces,"63,5 m²",4ème étage,2 chambres,NaN,NaN
3,seloger,25IIW65P1F9X,2025-10-29T16:55:44.0438727Z,2025-10-29T16:55:44.0438708Z,Lyon,69005,FR,Point,"[4.827683999999999, 45.76254799999993]",Particulier loue F3 bis de 85 m² rue St Jean d...,ST JEAN 85m² grand T3 bis bcp cachet sans vis ...,Appartement à louer,"['3 pièces', '2 chambres', '85 m²', 'Étage 1/4']",1 280 €,3 pièces,85 m²,Étage 1/4,2 chambres,NaN,NaN
4,seloger,25U9XDRKUJ3L,2025-07-04T09:35:00Z,2025-08-09T16:36:10.882Z,Lyon 1er,69001,FRA,MultiPolygon,"[[[[4.8269, 45.77195], [4.828, 45.77154], [4.8...",LYON 4E ARRONDISSEMENT- A LOUER Studio de 20m²...,Appartement - Studio - LYON 4E ARRONDISSEMENT ...,Appartement à louer,"['1 pièce', '20 m²']",560 €,1 pièce,20 m²,NaN,NaN,NaN,NaN


In [5]:
df.describe() 

,brand,id,metadata.creationDate,metadata.updateDate,sections.location.address.city,sections.location.address.zipCode,sections.location.address.country,sections.location.geometry.type,sections.location.geometry.coordinates,sections.description.description,sections.description.headline,sections.hardFacts.title,sections.hardFacts.keyfacts,sections.hardFacts.price.value,numberOfRooms,livingSpace,numberOfFloors,numberOfBedrooms,availability,plotSpace
count,3355,3355,3355,3355,3353,3355,3355,3355,3355,3355,3355,3355,3354,3355,3339,3340,3009,2497,374,10
unique,1,3355,2561,3352,33,34,2,2,2359,3310,2191,11,2793,1218,11,671,140,9,72,9
top,seloger,25EC9GDFBT55,2025-10-08T15:11:00Z,2025-11-20T13:50:10.47Z,Lyon 3ème,69003,FRA,Point,"[[[[4.8269, 45.77195], [4.828, 45.77154], [4.8...",## Space\r\n\r\n Nichée au 3 € étage d'un gran...,À propos de ce bien,Appartement à louer,"['1 pièce', '15 m²', '1er étage']",1 800 €,2 pièces,30 m²,1er étage,1 chambre,dès le 01/12/2025,760 m² de terrain
freq,3355,1,39,2,301,321,3206,1923,62,4,147,2983,10,40,1057,65,384,1200,33,2


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3355 entries, 0 to 3354
Data columns (total 20 columns):
 #   Column                                  Non-Null Count  Dtype 
---  ------                                  --------------  ----- 
 0   brand                                   3355 non-null   object
 1   id                                      3355 non-null   object
 2   metadata.creationDate                   3355 non-null   object
 3   metadata.updateDate                     3355 non-null   object
 4   sections.location.address.city          3353 non-null   object
 5   sections.location.address.zipCode       3355 non-null   object
 6   sections.location.address.country       3355 non-null   object
 7   sections.location.geometry.type         3355 non-null   object
 8   sections.location.geometry.coordinates  3355 non-null   object
 9   sections.description.description        3355 non-null   object
 10  sections.description.headline           3355 non-null   object
 11  sect

In [7]:
df.drop(["availability" , "plotSpace"],axis=1,inplace=True)

In [8]:
df

,brand,id,metadata.creationDate,metadata.updateDate,sections.location.address.city,sections.location.address.zipCode,sections.location.address.country,sections.location.geometry.type,sections.location.geometry.coordinates,sections.description.description,sections.description.headline,sections.hardFacts.title,sections.hardFacts.keyfacts,sections.hardFacts.price.value,numberOfRooms,livingSpace,numberOfFloors,numberOfBedrooms
0,seloger,25EC9GDFBT55,2025-11-20T08:56:00Z,2025-11-23T01:16:54.498Z,Paris 6ème,75006,FRA,Point,"[2.3387919, 48.8536809]",## Space\r\n\r\nThis accommodation is availabl...,Appartement de 34m2,Appartement à louer,"['1 pièce', '34 m²', '1er étage']",2 800 €,1 pièce,34 m²,1er étage,NaN
1,seloger,2524YZ9M2KYE,2025-11-06T14:14:00Z,2025-11-19T17:36:08.608Z,Lyon 5ème,69005,FRA,Point,"[4.794783, 45.758385]","26 B, rue des Aqueducs 69005 Lyon, au cœur du ...","Location F4 - 26 B, rue des Aqueducs 69005 Lyon",Appartement à louer,"['4 pièces', '3 chambres', '91,7 m²', 'Étage 7...",1 666 €,4 pièces,"91,7 m²",Étage 7/7,3 chambres
2,seloger,24KPL52W4HXP,2024-03-05T14:14:00Z,2025-10-30T01:20:07.218Z,Lyon 6ème,69006,FRA,Point,"[4.844802, 45.768051]",Au coeur du quartier FOCH - au 17 rue de Sèze ...,Appartement 3 pièces,Appartement à louer,"['3 pièces', '2 chambres', '63,5 m²', '4ème ét...",1 440 €,3 pièces,"63,5 m²",4ème étage,2 chambres
3,seloger,25IIW65P1F9X,2025-10-29T16:55:44.0438727Z,2025-10-29T16:55:44.0438708Z,Lyon,69005,FR,Point,"[4.827683999999999, 45.76254799999993]",Particulier loue F3 bis de 85 m² rue St Jean d...,ST JEAN 85m² grand T3 bis bcp cachet sans vis ...,Appartement à louer,"['3 pièces', '2 chambres', '85 m²', 'Étage 1/4']",1 280 €,3 pièces,85 m²,Étage 1/4,2 chambres
4,seloger,25U9XDRKUJ3L,2025-07-04T09:35:00Z,2025-08-09T16:36:10.882Z,Lyon 1er,69001,FRA,MultiPolygon,"[[[[4.8269, 45.77195], [4.828, 45.77154], [4.8...",LYON 4E ARRONDISSEMENT- A LOUER Studio de 20m²...,Appartement - Studio - LYON 4E ARRONDISSEMENT ...,Appartement à louer,"['1 pièce', '20 m²']",560 €,1 pièce,20 m²,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3350,seloger,25MTPA1TPQAW,2025-11-14T14:47:00Z,2025-11-14T14:51:05.416Z,Paris 18ème,75018,FRA,Point,"[2.3358418, 48.8904358]",Oiko Gestion vous propose un superbe studio de...,Superbe studio de 31m²,Appartement à louer,"['1 pièce', '31 m²', '2ème étage']",1 128 €,1 pièce,31 m²,2ème étage,NaN
3351,seloger,256AGUAPAE7X,2025-09-22T14:09:00Z,2025-11-13T12:31:16.496Z,Lyon 5ème,69005,FRA,MultiPolygon,"[[[[4.80239, 45.7517], [4.79941, 45.75046], [4...",Place en colocation\nAu coeur du quartier Poin...,Chambre en colocation- All Inclusive - Point d...,Appartement à louer,"['4 pièces', '1 chambre', '62 m²', 'RDC']",480 €,4 pièces,62 m²,RDC,1 chambre
3352,seloger,25BTJAKTZB2R,2025-11-20T07:41:00Z,2025-11-20T07:46:12.656Z,Paris 16ème,75016,FRA,Point,"[2.28324, 48.856]",L'Immobilière de Beauséjour vous propose en ex...,2 pièces • 55 m² • Paris 16ème • Jean Bologne,Appartement à louer,"['2 pièces', '1 chambre', '55 m²', '3ème étage']",2 295 €,2 pièces,55 m²,3ème étage,1 chambre
3353,seloger,256JGJP5HC47,2025-11-14T12:47:00Z,2025-11-14T12:51:05.087Z,Lyon 3ème,69003,FRA,MultiPolygon,"[[[[4.89486, 45.74378], [4.89188, 45.7391], [4...",Réf 414.10 - F2 - 9 Cours Richard Vitton 69003...,VITTON 9,Appartement à louer,"['2 pièces', '1 chambre', '57,4 m²', '2ème éta...",775 €,2 pièces,"57,4 m²",2ème étage,1 chambre


In [9]:
df = df.rename(columns={"metadata.creationDate": "creation_date",
    "metadata.updateDate": "update_date",
    "sections.location.address.city": "city",
    "sections.location.address.zipCode": "zip_code",
    "sections.location.address.country": "country",
    "sections.location.geometry.type": "geometry_type",
    "sections.location.geometry.coordinates": "geometry_coords",
    "sections.description.description": "description",
    "sections.description.headline": "headline",
    "sections.hardFacts.title": "title",
    "sections.hardFacts.keyfacts": "keyfacts",
    "sections.hardFacts.facts": "facts",
    "sections.hardFacts.price.value": "price_value"})

In [10]:
df.head()

,brand,id,creation_date,update_date,city,zip_code,country,geometry_type,geometry_coords,description,headline,title,keyfacts,price_value,numberOfRooms,livingSpace,numberOfFloors,numberOfBedrooms
0,seloger,25EC9GDFBT55,2025-11-20T08:56:00Z,2025-11-23T01:16:54.498Z,Paris 6ème,75006,FRA,Point,"[2.3387919, 48.8536809]",## Space\r\n\r\nThis accommodation is availabl...,Appartement de 34m2,Appartement à louer,"['1 pièce', '34 m²', '1er étage']",2 800 €,1 pièce,34 m²,1er étage,NaN
1,seloger,2524YZ9M2KYE,2025-11-06T14:14:00Z,2025-11-19T17:36:08.608Z,Lyon 5ème,69005,FRA,Point,"[4.794783, 45.758385]","26 B, rue des Aqueducs 69005 Lyon, au cœur du ...","Location F4 - 26 B, rue des Aqueducs 69005 Lyon",Appartement à louer,"['4 pièces', '3 chambres', '91,7 m²', 'Étage 7...",1 666 €,4 pièces,"91,7 m²",Étage 7/7,3 chambres
2,seloger,24KPL52W4HXP,2024-03-05T14:14:00Z,2025-10-30T01:20:07.218Z,Lyon 6ème,69006,FRA,Point,"[4.844802, 45.768051]",Au coeur du quartier FOCH - au 17 rue de Sèze ...,Appartement 3 pièces,Appartement à louer,"['3 pièces', '2 chambres', '63,5 m²', '4ème ét...",1 440 €,3 pièces,"63,5 m²",4ème étage,2 chambres
3,seloger,25IIW65P1F9X,2025-10-29T16:55:44.0438727Z,2025-10-29T16:55:44.0438708Z,Lyon,69005,FR,Point,"[4.827683999999999, 45.76254799999993]",Particulier loue F3 bis de 85 m² rue St Jean d...,ST JEAN 85m² grand T3 bis bcp cachet sans vis ...,Appartement à louer,"['3 pièces', '2 chambres', '85 m²', 'Étage 1/4']",1 280 €,3 pièces,85 m²,Étage 1/4,2 chambres
4,seloger,25U9XDRKUJ3L,2025-07-04T09:35:00Z,2025-08-09T16:36:10.882Z,Lyon 1er,69001,FRA,MultiPolygon,"[[[[4.8269, 45.77195], [4.828, 45.77154], [4.8...",LYON 4E ARRONDISSEMENT- A LOUER Studio de 20m²...,Appartement - Studio - LYON 4E ARRONDISSEMENT ...,Appartement à louer,"['1 pièce', '20 m²']",560 €,1 pièce,20 m²,NaN,NaN


In [11]:
df.isna().sum()

brand                 0
id                    0
creation_date         0
update_date           0
city                  2
zip_code              0
country               0
geometry_type         0
geometry_coords       0
description           0
headline              0
title                 0
keyfacts              1
price_value           0
numberOfRooms        16
livingSpace          15
numberOfFloors      346
numberOfBedrooms    858
dtype: int64

In [12]:
df[df["livingSpace"].isnull()]

,brand,id,creation_date,update_date,city,zip_code,country,geometry_type,geometry_coords,description,headline,title,keyfacts,price_value,numberOfRooms,livingSpace,numberOfFloors,numberOfBedrooms
181,seloger,25NS1GNE8X2E,2025-07-04T17:04:00Z,2025-09-09T17:05:09.531Z,Paris 8ème,75008,FRA,MultiPolygon,"[[[[2.32093, 48.86302], [2.30174, 48.8635], [2...",LOCATION APPARTEMENT MEUBLE - 2 PIÈCES - PARIS...,Location Maison Meublée PARIS 75008,Maison à louer,"['2 pièces', '1 chambre']",2 300 €,2 pièces,NaN,NaN,1 chambre
606,seloger,25238VASYXP4,2025-11-20T20:12:00Z,2025-11-21T05:46:06.276Z,Lyon 3ème,69003,FRA,Point,"[4.8729941, 45.7571181]",Logement Manda ! Visitez et déposez votre doss...,Lyon,Studio à louer,"['4 pièces', '3 chambres', 'RDC']",550 €,4 pièces,NaN,RDC,3 chambres
1000,seloger,25HZ4K3CVWFW,2025-09-25T19:31:00Z,2025-11-06T21:36:14.716Z,Lyon 8ème,69008,FRA,Point,"[4.8838482, 45.7223654]",Logement Manda ! Visitez et déposez votre doss...,?? Chambre meublée en colocation moderne – Lyo...,Studio à louer,"['6 pièces', '5 chambres', '7ème étage']",466 €,6 pièces,NaN,7ème étage,5 chambres
1167,seloger,23GGAYAQZRIY,2023-08-21T09:01:00Z,2025-07-16T08:35:12.009Z,Lyon 5ème,69005,FRA,Point,"[4.786041, 45.756849]","Dans une résidence familiale, appartement réno...",À propos de ce bien,Appartement à louer,"['4 pièces', '3 chambres']",525 €,4 pièces,NaN,NaN,3 chambres
1311,seloger,25R5TNUZTLJ7,2025-11-20T09:36:00Z,2025-11-22T08:40:07.677Z,Paris 8ème,75008,FRA,Point,"[2.299989, 48.876917]",## Space\r\nCe logement est disponible pour un...,Appartement de 11m2,Appartement à louer,"['2 pièces', '6ème étage', 'dès le 23/12/2025']",1 350 €,2 pièces,NaN,6ème étage,NaN
1535,seloger,25D7SZ439356,2025-11-13T15:18:00Z,2025-11-18T09:06:14.31Z,Lyon 4ème,69004,FRA,MultiPolygon,"[[[[4.84132, 45.77947], [4.84306, 45.77878], [...",Votre agence LES CLÉS D'ALEXIA vous propose u...,Colocation meublée pour 3 personnes – Caluire ...,Appartement à louer,"['4 pièces', '3 chambres', 'Étage 4/4']",500 €,4 pièces,NaN,Étage 4/4,3 chambres
1543,seloger,25CI3KMDDFTC,2025-11-17T22:02:00Z,2025-11-22T22:01:19.333Z,Lyon 5ème,69005,FRA,MultiPolygon,"[[[[4.81266, 45.74837], [4.81111, 45.7476], [4...",A louer appartement de type T3 avec balcon de ...,RUE DES CHEVAUCHEURS funiculaire ST JUST - T...,Appartement à louer,"['3 pièces', '2 chambres', 'Étage 1/2']",1 100 €,3 pièces,NaN,Étage 1/2,2 chambres
1708,seloger,25RYJZDDF6F1,2025-11-12T17:44:00Z,2025-11-12T17:46:14.27Z,Lyon 4ème,69004,FRA,MultiPolygon,"[[[[4.82928, 45.78293], [4.82929, 45.78293], [...",Particulier loue duplex meublé Croix Rousse.\r...,T2 Meuble Croix Rousse,Appartement à louer,"['2 pièces', '1 chambre', 'RDC']",885 €,2 pièces,NaN,RDC,1 chambre
1813,seloger,25UY424KVTZ9,2025-11-18T15:44:00Z,2025-11-22T21:05:06.045Z,Lyon 8ème,69008,FRA,Point,"[4.852754116, 45.729816437]","Au sein d'une copropriété, découvrez ce bel ap...",CHAMBRE COLOCATION - 69008 - 11m²,Appartement à louer,"['5 pièces', '1 chambre', '2ème étage', 'dès l...",600 €,5 pièces,NaN,2ème étage,1 chambre
2575,seloger,251MGVC92B1D,2025-07-09T12:04:00Z,2025-08-09T16:36:10.843Z,Lyon 1er,69001,FRA,MultiPolygon,"[[[[4.83963, 45.76861], [4.83976, 45.76627], [...",LYON 2E ARRONDISSEMENT - A LOUER Chambre Loyer...,LYON 2EME CHAMBRE 800€ CC,Appartement à louer,"['1 chambre', 'Étage 4/5']",800 €,NaN,NaN,Étage 4/5,1 chambre


In [13]:
df.shape[0]

3355

In [14]:
df = df.dropna(subset=['livingSpace'])


In [15]:
df.isna().sum()

brand                 0
id                    0
creation_date         0
update_date           0
city                  2
zip_code              0
country               0
geometry_type         0
geometry_coords       0
description           0
headline              0
title                 0
keyfacts              0
price_value           0
numberOfRooms        13
livingSpace           0
numberOfFloors      340
numberOfBedrooms    854
dtype: int64

In [16]:
df.shape[0]

3340

In [17]:
df['creation_date'] = pd.to_datetime(df['creation_date'], errors='coerce')
df['update_date']   = pd.to_datetime(df['update_date'], errors='coerce')

In [18]:
df["geometry_type"].unique()


array(['Point', 'MultiPolygon'], dtype=object)

In [19]:
def get_centroid(row):
    coords = row["geometry_coords"]

    # Convert string to Python object
    if isinstance(coords, str):
        try:
            coords = ast.literal_eval(coords)
        except:
            return pd.Series([np.nan, np.nan])

    # Recréer un dict GeoJSON minimal
    geom = {"type": row["geometry_type"], "coordinates": coords}

    # Shapely gère tout seul Point / Polygon / MultiPolygon
    try:
        centroid = shape(geom).centroid
        return pd.Series([centroid.x, centroid.y])
    except:
        return pd.Series([np.nan, np.nan])


In [20]:
df[['lon', 'lat']] = df.apply(get_centroid, axis=1)

In [21]:
cat_cols = [
    'brand', 'city', 'zip_code', 'country', 'geometry_type',
    'title', 'headline'
]

df[cat_cols] = df[cat_cols].astype('category')


In [22]:
def clean_numeric(col):
    return (
        col.astype(str)
           .str.replace(r"[^\d,\.]", "", regex=True)  # garde chiffres ., 
           .str.replace(",", ".", regex=False)        # virgule -> .
           .replace("", np.nan)                        
           .replace("nan", np.nan)                    # au cas où
           .astype(float)
    )


In [23]:
num_cols = [
    'price_value',
    'numberOfRooms',
    'livingSpace',
    'numberOfFloors',
    'numberOfBedrooms'
]

df[num_cols] = df[num_cols].apply(clean_numeric)


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3340 entries, 0 to 3354
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   brand             3340 non-null   category           
 1   id                3340 non-null   object             
 2   creation_date     3196 non-null   datetime64[ns, UTC]
 3   update_date       3339 non-null   datetime64[ns, UTC]
 4   city              3338 non-null   category           
 5   zip_code          3340 non-null   category           
 6   country           3340 non-null   category           
 7   geometry_type     3340 non-null   category           
 8   geometry_coords   3340 non-null   object             
 9   description       3340 non-null   object             
 10  headline          3340 non-null   category           
 11  title             3340 non-null   category           
 12  keyfacts          3340 non-null   object             
 13  price_va

In [25]:
df[df["city"].isna()]

,brand,id,creation_date,update_date,city,zip_code,country,geometry_type,geometry_coords,description,headline,title,keyfacts,price_value,numberOfRooms,livingSpace,numberOfFloors,numberOfBedrooms,lon,lat
2271,seloger,25IV7MFML213,NaT,2025-11-21 13:09:23.579433500+00:00,NaN,75013,FR,Point,"[2.362258961566145, 48.828422056819065]","À louer, superbe appartement 3 pièces neuf sit...","Superbe 3 pièces neuf avec grande terrasse, pa...",Appartement à louer,"['3 pièces', '2 chambres', '68 m²', 'Étage 6/15']",2250.0,3.0,68.0,615.0,2.0,2.362259,48.828422
2894,seloger,251V5PVZ5AIT,NaT,2025-10-23 22:13:39.122583300+00:00,NaN,69008,FR,Point,"[4.8689454997582855, 45.733910870477764]","Appartement T4 d'une superficie de 100m2, situ...",T4 100 m2 dans résidence calme avec garage et ...,Appartement à louer,"['4 pièces', '3 chambres', '100 m²', 'Étage 2/3']",1350.0,4.0,100.0,23.0,3.0,4.868945,45.733911


In [26]:
def infer_city(zip_code):
    zip_code = str(zip_code)
    if zip_code.startswith("75"):
        return "Paris"
    if zip_code.startswith("69"):
        return "Lyon"
    return None
df["city"] = df["zip_code"].apply(infer_city)
df = df[df["city"].notna()]


In [27]:
df[df["city"].isna()]

,brand,id,creation_date,update_date,city,zip_code,country,geometry_type,geometry_coords,description,headline,title,keyfacts,price_value,numberOfRooms,livingSpace,numberOfFloors,numberOfBedrooms,lon,lat


In [28]:
df["price_m2"] = (df["price_value"] / df["livingSpace"]).round(0)

In [29]:
df["price_m2"].isna().sum()

np.int64(0)

In [30]:
df.head()

,brand,id,creation_date,update_date,city,zip_code,country,geometry_type,geometry_coords,description,...,title,keyfacts,price_value,numberOfRooms,livingSpace,numberOfFloors,numberOfBedrooms,lon,lat,price_m2
0,seloger,25EC9GDFBT55,2025-11-20 08:56:00+00:00,2025-11-23 01:16:54.498000+00:00,Paris,75006,FRA,Point,"[2.3387919, 48.8536809]",## Space\r\n\r\nThis accommodation is availabl...,...,Appartement à louer,"['1 pièce', '34 m²', '1er étage']",2800.0,1.0,34.0,1.0,NaN,2.338792,48.853681,82.0
1,seloger,2524YZ9M2KYE,2025-11-06 14:14:00+00:00,2025-11-19 17:36:08.608000+00:00,Lyon,69005,FRA,Point,"[4.794783, 45.758385]","26 B, rue des Aqueducs 69005 Lyon, au cœur du ...",...,Appartement à louer,"['4 pièces', '3 chambres', '91,7 m²', 'Étage 7...",1666.0,4.0,91.7,77.0,3.0,4.794783,45.758385,18.0
2,seloger,24KPL52W4HXP,2024-03-05 14:14:00+00:00,2025-10-30 01:20:07.218000+00:00,Lyon,69006,FRA,Point,"[4.844802, 45.768051]",Au coeur du quartier FOCH - au 17 rue de Sèze ...,...,Appartement à louer,"['3 pièces', '2 chambres', '63,5 m²', '4ème ét...",1440.0,3.0,63.5,4.0,2.0,4.844802,45.768051,23.0
3,seloger,25IIW65P1F9X,NaT,2025-10-29 16:55:44.043870800+00:00,Lyon,69005,FR,Point,"[4.827683999999999, 45.76254799999993]",Particulier loue F3 bis de 85 m² rue St Jean d...,...,Appartement à louer,"['3 pièces', '2 chambres', '85 m²', 'Étage 1/4']",1280.0,3.0,85.0,14.0,2.0,4.827684,45.762548,15.0
4,seloger,25U9XDRKUJ3L,2025-07-04 09:35:00+00:00,2025-08-09 16:36:10.882000+00:00,Lyon,69001,FRA,MultiPolygon,"[[[[4.8269, 45.77195], [4.828, 45.77154], [4.8...",LYON 4E ARRONDISSEMENT- A LOUER Studio de 20m²...,...,Appartement à louer,"['1 pièce', '20 m²']",560.0,1.0,20.0,NaN,NaN,4.829501,45.770835,28.0


In [31]:
df.to_csv("data/cleaned.csv", index=False)

In [ ]:
df["creation_date"].min()

<DatetimeArray>
['2025-11-20 08:56:00+00:00', '2025-11-06 14:14:00+00:00',
 '2024-03-05 14:14:00+00:00',                       'NaT',
 '2025-07-04 09:35:00+00:00', '2025-09-04 15:07:00+00:00',
 '2025-11-19 06:12:00+00:00', '2025-11-17 16:12:00+00:00',
 '2025-10-24 00:04:00+00:00', '2025-11-17 19:06:00+00:00',
 ...
 '2021-04-07 13:34:00+00:00', '2025-11-18 23:41:00+00:00',
 '2025-11-19 10:28:00+00:00', '2023-05-10 09:04:00+00:00',
 '2025-09-25 06:14:00+00:00', '2025-11-21 06:48:00+00:00',
 '2025-11-14 14:47:00+00:00', '2025-09-22 14:09:00+00:00',
 '2025-11-20 07:41:00+00:00', '2025-11-14 12:47:00+00:00']
Length: 2411, dtype: datetime64[ns, UTC]

In [ ]:
# TODO: finir de cleaner -> supprimer les doublons , checker les unités , écrire les nombres avec des regex
# TODO: structurer le projet 
# TODO: faire des graphs -> histogramme avec tous les points des différents m2 et une droite moyenne + une droite médiane
# TODO: app streamlit
# TODO: régler les NaT
# TODO: un histogramme avec les dates 
# TODO : Boite a moustache avec les moyenne mediane 
# TODO : moyenne mediane affichee a cote
# TODO : prioriser le fait de pouvoir scrap plus de villes
# TODO : eviter les doublons
